In [ ]:
import pandas as pd
import numpy as np
print('nsvm;veve')

In [ ]:
#функцмя для импутирования среднего по классам
from sklearn.impute import SimpleImputer
def mean_by_class(df):
    df_by_group=df.groupby('label')
    my_dict={}
    my_imputer=SimpleImputer(missing_values=np.nan, strategy='mean')
    for group,data in df_by_group:
        imput_data=pd.DataFrame(my_imputer.fit_transform(data))
        imput_data[90]=imput_data[90].astype(int)
        imput_data.rename(columns={90: 'label'}, inplace=True)
        my_dict[group]=imput_data
    res_df=pd.concat([my_dict[key] for key in my_dict.keys()],ignore_index=True)
    return res_df

In [ ]:
# input train data
df=pd.read_csv('train.csv')
###
#пробуем просто скипнуть данные с nan
# df.dropna(how='any',inplace=True)
###
print(df.shape)
df.drop(['id'], axis=1, inplace=True)
#######
#пробуем имутировать данные по классам
# df=mean_by_class(df) # не дало результатов
######
df_Y_tr=df['label']
df_X_tr=df.drop(['label'],axis=1, inplace=False)
# смотрим глазами на данные
print('mean:  ',*df.mean())
print('var:  ',*df.var())
df.corr().to_excel('corr_matrix.xlsx')
df

In [19]:
# input test data
df_0=pd.read_csv('test.csv')
df_X_tst=df_0.drop(['id'], axis=1, inplace=False)

In [7]:
# 1 strategy of inputing:  simpleinputing, mean
# from sklearn.impute import SimpleImputer
# my_simple_imputer=SimpleImputer( strategy='mean')
# X_tr=my_simple_imputer.fit_transform(df_X_tr)
# X_tst=my_simple_imputer.fit_transform(df_X_tst)

In [20]:
# пробуем вставить расширеную матрицу с метками о вставках
from sklearn.impute import MissingIndicator
my_miss_ind=MissingIndicator()
mis_df_tr=my_miss_ind.fit_transform(df_X_tr).astype('int')
mis_df_tst=my_miss_ind.fit_transform(df_X_tst).astype('int')
print(mis_df_tr.shape,mis_df_tst.shape)
mis_df_tr

(50000, 15) (10000, 15)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]])

In [21]:
# # 2 strategy of imputing: KNNInputing
from sklearn.impute import KNNImputer
my_simple_imputer=KNNImputer(n_neighbors=15, weights='distance')
X_tr=my_simple_imputer.fit_transform(df_X_tr)
X_tst=my_simple_imputer.fit_transform(df_X_tst)

#knn imputer работает лучше

In [22]:
# пробуем соединить
X_tr=np.concatenate((X_tr, mis_df_tr), axis=1)
X_tst=np.concatenate((X_tst, mis_df_tst), axis=1)
print(X_tr.shape,X_tst.shape)# лучше с матрицей меток

(50000, 105) (10000, 105)


In [23]:
# to M=0 and D=1
from sklearn.preprocessing import StandardScaler
standarter=StandardScaler()
st_X_tr=standarter.fit_transform(X_tr)
st_X_tst=standarter.fit_transform(X_tst)

In [24]:
# split to train_val
from sklearn.model_selection import train_test_split
Y_tr=np.array(df_Y_tr)
X_train, X_val, y_train, y_val=train_test_split(st_X_tr,Y_tr,test_size=0.2,random_state=42,shuffle=True)


In [12]:
# fit and predict model(KNN)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,roc_auc_score
neighbors=list(range(26,89))
weights=['distance','uniform']
p_list=[1,2]
for n_neighbors in neighbors:
    for weight in weights:
        for p in p_list:
            my_knn=KNeighborsClassifier(n_neighbors=n_neighbors,weights=weight,p=p)
            my_knn.fit(X_train,y_train)
            y_pred_val=my_knn.predict_proba(X_val)

            print(f"roc_auc={roc_auc_score(y_val,y_pred_val,multi_class='ovr')} with n_neighbors={n_neighbors}, weights={weight},p={p}")#7-26: max roc_auc=0.9651984308882959 with n_neighbors=37, weights=distance,p=1

KeyboardInterrupt: 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score
my_knn_test=KNeighborsClassifier(n_neighbors=37,weights='distance',p=1)
my_knn_test.fit(X_train,y_train)
y_pred_val_knn_min=my_knn_test.predict(X_val)
print(accuracy_score(y_pred_val_knn_min,y_val))
# print(recall_score(y_pred_val_knn_min,y_val))
# print(precision_score(y_pred_val_knn_min,y_val))
print('best_acc=0.7412, n_n=37,w=distance,p=1')

0.7412
best_acc=0.74, n_n=16,w=distance,p=1


In [ ]:
#пробуем кнн с косинусным расстояние
from sklearn.metrics import roc_auc_score
n_list=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # max acc=0.96829 при n=20, weight=dist
big_n_list=[21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40] # max acc=0.9707 при n= 36, weight=dist
bigbig_n_list=range(40,89)# max roc_auc= 0.9706< 0.759
my_list=[]
#distance
for n in bigbig_n_list:
    my_knn_cos=KNeighborsClassifier(n_neighbors=n,metric='cosine',weights='distance')
    my_knn_cos.fit(X_train,y_train)
    y_pred_val_knn_cos=my_knn_cos.predict_proba(X_val)
    acc=roc_auc_score(y_val,y_pred_val_knn_cos,multi_class='ovo')
    acc1=roc_auc_score(y_val,y_pred_val_knn_cos,multi_class='ovr')
    print(acc,'n=',n,'distance')
    print('acc1=',acc1,'n=',n,'distance')
    my_list.append((acc,n))
print(sorted(my_list, key=lambda x: x[0],reverse=True)[0])
#uniform
for n in bigbig_n_list:
    my_knn_cos=KNeighborsClassifier(n_neighbors=n,metric='cosine',weights='uniform')
    my_knn_cos.fit(X_train,y_train)
    y_pred_val_knn_cos=my_knn_cos.predict_proba(X_val)
    acc=roc_auc_score(y_val,y_pred_val_knn_cos,multi_class='ovo')
    acc1=roc_auc_score(y_val,y_pred_val_knn_cos,multi_class='ovr')
    print(acc,'n=',n,'uniform')
    print('acc1=',acc1,'n=',n,'uniform')
    my_list.append((acc,n))
print(sorted(my_list, key=lambda x: x[0],reverse=True)[0])

0.9706034744299064 n= 40 distance
acc1= 0.9707383665809324 n= 40 distance
0.9705555228016393 n= 41 distance
acc1= 0.9706911608203539 n= 41 distance
0.970587429648759 n= 42 distance
acc1= 0.970723211187185 n= 42 distance
0.9705986963526096 n= 43 distance
acc1= 0.9707324073278434 n= 43 distance
0.9705472367916311 n= 44 distance
acc1= 0.9706826033008433 n= 44 distance
0.9704924591858107 n= 45 distance
acc1= 0.9706306466896806 n= 45 distance
0.9705123941309667 n= 46 distance
acc1= 0.9706512889154068 n= 46 distance
0.9705001384671755 n= 47 distance
acc1= 0.9706349412201319 n= 47 distance
0.9704870618059702 n= 48 distance
acc1= 0.9706246685260069 n= 48 distance
0.9704131705294206 n= 49 distance
acc1= 0.9705524343412517 n= 49 distance
0.9704039261023564 n= 50 distance
acc1= 0.970545295933055 n= 50 distance
0.9703169131635979 n= 51 distance
acc1= 0.9704540311037423 n= 51 distance
0.9703294784804214 n= 52 distance
acc1= 0.9704647388966687 n= 52 distance
0.9703273913976516 n= 53 distance
acc1= 0

KeyboardInterrupt: 

In [ ]:
#берем максимальное значение для косинусного расстояния
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
my_knn_cos_test=KNeighborsClassifier(n_neighbors=40,metric='cosine',weights='distance')
my_knn_cos_test.fit(X_train,y_train)
y_pred_val_knn_cos=my_knn_cos_test.predict(X_val)
acc=accuracy_score(y_pred_val_knn_cos,y_val)
print(acc)#0.7613

0.7696


In [ ]:
#решающие деревья


In [ ]:
# fit and predict model(GNB)
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.metrics import accuracy_score,roc_auc_score
my_GNB=GaussianNB() #GNB, acc=0.67
my_GNB.fit(X_train,y_train)
y_pred_val_GNB=my_GNB.predict_proba(X_val)

print('roc_auc=',roc_auc_score(y_val,y_pred_val_GNB, multi_class='ovr'))
#не удачно, кнн дает больше

roc_auc= 0.9064123005890785


In [ ]:
# пробуем решающие деревья
from sklearn.tree import DecisionTreeClassifier
criterion_list=['gini','entropy','log_loss']
max_d=list(range(1,20))
for criterion in criterion_list:
    for d in max_d:
        my_klassifire_tree=DecisionTreeClassifier(criterion=criterion, max_depth=d)
        my_klassifire_tree.fit(X_train,y_train)
        y_pred_val_tree=my_klassifire_tree.predict_proba(X_val)
        print(f"criterion:{criterion}, max_depth={d}, accuracy={roc_auc_score(y_val,y_pred_val_tree,multi_class='ovr')}")# max criterion:log_loss, max_depth=8, roc_auc=0.9172528776609592

criterion:gini, max_depth=1, accuracy=0.5782538880448167
criterion:gini, max_depth=2, accuracy=0.6974788595940817
criterion:gini, max_depth=3, accuracy=0.7507541056493364
criterion:gini, max_depth=4, accuracy=0.8154416558707036
criterion:gini, max_depth=5, accuracy=0.8535607663743177
criterion:gini, max_depth=6, accuracy=0.8794051955105173
criterion:gini, max_depth=7, accuracy=0.8959983204009803
criterion:gini, max_depth=8, accuracy=0.9052157831098308
criterion:gini, max_depth=9, accuracy=0.9073302010031135
criterion:gini, max_depth=10, accuracy=0.9087543636268721
criterion:gini, max_depth=11, accuracy=0.9054417468364628
criterion:gini, max_depth=12, accuracy=0.8920877403488344
criterion:gini, max_depth=13, accuracy=0.8791603951790901
criterion:gini, max_depth=14, accuracy=0.861735901680911
criterion:gini, max_depth=15, accuracy=0.8482377458924109
criterion:gini, max_depth=16, accuracy=0.8321334493556407
criterion:gini, max_depth=17, accuracy=0.817828336936502
criterion:gini, max_depth

In [ ]:
# пробуем svm
from sklearn.svm import SVC
# classifire_list=[SVC,LinearSVC,NuSVC]
ker_list=['linear', 'poly', 'rbf', 'sigmoid']
degree_list=[2,3,4,5,6]
for ker in ker_list:
    if ker== 'poly':
        for degree in degree_list:
            my_SVM_classifire=SVC(kernel=ker,degree=degree,probability=True)
            my_SVM_classifire.fit(X_train,y_train)
            y_pred_val_SVM=my_SVM_classifire.predict_proba(X_val)
            print(f"SVC, ker={ker},degree_=={degree} roc_auc={roc_auc_score(y_val,y_pred_val_SVM,multi_class='ovr')}")# best: ker=poly,degree_==2 roc_auc=0.9733805417949194
    elif ker=='linear':
        my_SVM_classifire=SVC(kernel=ker,probability=True)
        my_SVM_classifire.fit(X_train,y_train)
        y_pred_val_SVM=my_SVM_classifire.predict_proba(X_val)
        print(f"SVC, ker={ker} roc_auc={roc_auc_score(y_val,y_pred_val_SVM,multi_class='ovr')}")#roc_auc=0.9579
    elif ker=='rbf':
        my_SVM_classifire=SVC(kernel=ker,probability=True)
        my_SVM_classifire.fit(X_train,y_train)
        y_pred_val_SVM=my_SVM_classifire.predict_proba(X_val)
        print(f"SVC, ker={ker} roc_auc={roc_auc_score(y_val,y_pred_val_SVM,multi_class='ovr')}")#ker=rbf roc_auc=0.9742930886535897
    else:
        my_SVM_classifire=SVC(kernel=ker,probability=True)
        my_SVM_classifire.fit(X_train,y_train)
        y_pred_val_SVM=my_SVM_classifire.predict_proba(X_val)
        print(f"SVC, ker={ker} roc_auc={roc_auc_score(y_val,y_pred_val_SVM,multi_class='ovr')}")#ker=sigmoid roc_auc=0.9291839548505889

SVC, ker=linear roc_auc=0.9579413168303761
SVC, ker=poly,degree_==2 roc_auc=0.9733805417949194
SVC, ker=poly,degree_==3 roc_auc=0.9731969793348403
SVC, ker=poly,degree_==4 roc_auc=0.9731282969935678
SVC, ker=poly,degree_==5 roc_auc=0.9688603954030292
SVC, ker=poly,degree_==6 roc_auc=0.9627927690330195
SVC, ker=rbf roc_auc=0.9742930886535897
SVC, ker=sigmoid roc_auc=0.9291839548505889


In [28]:
#пробуем случайный лес, хоть я в этом ничего не понял
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
n_tree_list=list(range(400,1000,100))
depth_list=list(range(50,100,10))
for n_tree in n_tree_list:
    for depth in depth_list:
        my_ensamble_classifire=RandomForestClassifier(n_estimators=n_tree,criterion='log_loss',max_depth=depth)
        my_ensamble_classifire.fit(X_train,y_train)
        y_pred_val_ensamble=my_ensamble_classifire.predict_proba(X_val)
        print(f"n_treee:{n_tree}, max_depth={depth}, roc_auc={roc_auc_score(y_val,y_pred_val_ensamble,multi_class='ovr')}")#n_treee:440, max_depth=44, roc_auc=0.97046271153308

n_treee:400, max_depth=50, roc_auc=0.9673425919327698


KeyboardInterrupt: 

In [ ]:
#best of svm:
from sklearn.svm import SVC
my_SVM_classifire_test=SVC(kernel='rbf',probability=True)
my_SVM_classifire_test.fit(st_X_tr,Y_tr)
# my_SVM_classifire_test.fit(X_train,y_train)
# y_pred_val_SVM_test=my_SVM_classifire.predict_proba(X_val)
# print(roc_auc_score(y_val,y_pred_val_SVM,multi_class='ovr'))

SVC(probability=True)

In [ ]:
#best of DTC
from sklearn.tree import DecisionTreeClassifier
my_klassifire_tree_test=DecisionTreeClassifier(criterion='entropy', max_depth=13)
my_klassifire_tree_test.fit(st_X_tr,Y_tr)

DecisionTreeClassifier(criterion='entropy', max_depth=13)

In [ ]:
# пробуем объединить несколько предсказаний
from sklearn.metrics import accuracy_score,precision_score,recall_score

pred_val=(y_pred_val_GNB+y_pred_val_knn_cos+y_pred_val_knn_min)/3
print(*pred_val)
pred_val=np.round(pred_val)
print(accuracy_score(pred_val,y_val))

6.0 0.0 4.0 3.0 4.0 5.0 0.0 0.0 6.0 4.0 7.0 4.0 8.0 6.0 3.0 4.0 5.0 0.3333333333333333 4.0 1.3333333333333333 0.0 2.0 4.0 9.0 3.0 0.0 6.0 5.666666666666667 8.333333333333334 2.3333333333333335 1.0 1.0 0.0 3.0 1.0 0.0 3.0 1.0 2.0 7.0 3.6666666666666665 0.0 7.0 4.0 8.0 5.0 9.0 4.0 9.0 3.0 1.0 3.0 9.0 6.0 3.0 6.0 6.0 6.0 6.333333333333333 6.0 0.0 0.0 9.0 9.0 5.0 6.0 1.0 4.0 1.0 2.0 4.0 5.666666666666667 7.0 4.666666666666667 2.0 2.0 7.666666666666667 1.0 4.333333333333333 3.0 3.0 1.0 9.0 3.0 2.0 7.0 6.0 0.0 2.0 4.0 0.0 0.0 1.0 1.0 1.0 9.0 6.666666666666667 1.0 1.0 6.333333333333333 1.0 7.0 1.0 9.0 4.0 1.0 6.0 6.333333333333333 3.0 3.3333333333333335 9.0 4.0 6.333333333333333 0.0 1.0 6.0 3.0 6.0 3.0 4.0 3.0 5.0 1.0 4.0 4.0 2.0 0.0 6.0 8.0 1.0 6.0 3.0 6.333333333333333 3.0 2.6666666666666665 3.6666666666666665 4.0 4.0 3.0 5.666666666666667 0.0 1.0 8.0 5.0 5.666666666666667 9.0 1.0 0.0 2.0 6.666666666666667 0.0 7.0 1.0 3.0 3.0 4.0 4.0 1.0 7.0 5.0 7.0 5.333333333333333 1.6666666666666667 6.0 

In [25]:
#создаем все классификаторы для тупого усреднения
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
model_knn_min_p_1_n37=KNeighborsClassifier(n_neighbors=37,weights='distance',p=1)
# model_knn_min_p_1_n37.fit(st_X_tr,Y_tr)
model_knn_cos_n40=KNeighborsClassifier(n_neighbors=40,weights='distance',metric='cosine')
# model_knn_cos_n40.fit(st_X_tr,Y_tr)
model_GNB=GaussianNB()
# model_GNB.fit(st_X_tr,Y_tr)
model_tree_logloss_8=DecisionTreeClassifier(criterion='log_loss', max_depth=8)
# model_tree_logloss_8.fit(st_X_tr,Y_tr)
model_SVC_lin=SVC(kernel='linear',probability=True)
# model_SVC_lin.fit(st_X_tr,Y_tr)
model_SVC_poly_2=SVC(kernel='poly',degree=2,probability=True)
# model_SVC_poly_2.fit(st_X_tr,Y_tr)
model_SVC_rbf=SVC(kernel='rbf',probability=True)
# model_SVC_rbf.fit(st_X_tr,Y_tr)
model_forest_tree_440_d44=RandomForestClassifier(n_estimators=440,criterion='log_loss',max_depth=44)
# model_forest_tree_440_d44.fit(st_X_tr,Y_tr)
model_list=[
            ('gnb',model_GNB),
            ('knn1',model_knn_min_p_1_n37),
            ('knn2',model_knn_cos_n40),
            ('SVC_rbf',model_SVC_rbf),
            ('SVC_poly',model_SVC_poly_2),
            ('forest',model_forest_tree_440_d44)
            ]

In [ ]:
# для последующего запихивания в логрег
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
model_knn_min_p_1_n37=KNeighborsClassifier(n_neighbors=37,weights='distance',p=1)
model_knn_min_p_1_n37.fit(X_train,y_train)
model_knn_cos_n40=KNeighborsClassifier(n_neighbors=40,weights='distance',metric='cosine')
model_knn_cos_n40.fit(X_train,y_train)
model_GNB=GaussianNB()
model_GNB.fit(X_train,y_train)
model_tree_logloss_8=DecisionTreeClassifier(criterion='log_loss', max_depth=8)
model_tree_logloss_8.fit(X_train,y_train)
model_SVC_lin=SVC(kernel='linear',probability=True)
model_SVC_lin.fit(X_train,y_train)
model_SVC_poly_2=SVC(kernel='poly',degree=2,probability=True)
model_SVC_poly_2.fit(X_train,y_train)
model_SVC_rbf=SVC(kernel='rbf',probability=True)
model_SVC_rbf.fit(X_train,y_train)
model_forest_tree_440_d44=RandomForestClassifier(n_estimators=440,criterion='log_loss',max_depth=44)
model_forest_tree_440_d44.fit(X_train,y_train)

In [29]:
#  бэггинг hard and soft
from sklearn.ensemble import VotingClassifier

my_voiter_hard=VotingClassifier(model_list,voting='hard')
my_voiter_soft=VotingClassifier(model_list,voting='soft')
my_voiter_hard.fit(st_X_tr,Y_tr)
my_voiter_soft.fit(st_X_tr,Y_tr)
X_test=np.array(st_X_tst)
hard_pred=my_voiter_hard.predict(X_test)
soft_pred=my_voiter_soft.predict(X_test)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
X_test=np.array(st_X_tst)
my_classifire_stack=StackingClassifier(model_list,final_estimator=LogisticRegression())
my_classifire_stack.fit(X_train,y_train)
final_pred=my_classifire_stack.predict(X_val)
print(accuracy_score(final_pred,y_val))

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
X_test=np.array(st_X_tst)
my_classifire_stack=StackingClassifier(model_list,final_estimator=LogisticRegression())
my_classifire_stack.fit(st_X_tr,Y_tr)
final_pred_0=my_classifire_stack.predict(X_test)

In [ ]:
# пробуем скрестить методы
# X_test=np.array(st_X_tst)
# y_pred_knn_min=my_knn_test.predict(X_test)
# y_pred_knn_cos=my_knn_cos_test.predict(X_test)
# y_pred_GNB=my_GNB.predict(X_test)
# y_pred_tree=my_klassifire_tree_test.predict(X_test)
# y_pred_svm=my_SVM_classifire_test.predict(X_test)

df_out_hard = pd.DataFrame(final_pred_0, columns=['label'])
df_out_hard.to_csv('stack.csv', index=True, index_label='id')
# df_out_soft = pd.DataFrame(soft_pred, columns=['label'])
# df_out_soft.to_csv('soft_bagging_short.csv', index=True, index_label='id')
# auc 9905: knn, n=15.p=1,weights=1